## 소개

이 강의에서는 다음 내용을 다룹니다:
- 함수 호출이란 무엇이며, 어떤 경우에 사용하는지
- Azure OpenAI를 사용하여 함수 호출을 만드는 방법
- 함수 호출을 애플리케이션에 통합하는 방법

## 학습 목표

이 강의를 마치면 다음을 이해하고 할 수 있게 됩니다:

- 함수 호출을 사용하는 목적
- Azure Open AI 서비스에서 함수 호출 설정하기
- 애플리케이션의 사용 사례에 맞는 효과적인 함수 호출 설계하기


## 함수 호출 이해하기

이번 강의에서는 교육 스타트업을 위해 사용자가 챗봇을 통해 기술 강좌를 찾을 수 있는 기능을 만들어보려고 합니다. 사용자의 실력, 현재 역할, 관심 있는 기술에 맞는 강좌를 추천해줄 예정입니다.

이를 위해 다음을 조합해서 사용할 것입니다:
 - `Azure Open AI`를 이용해 사용자에게 챗 경험을 제공합니다
 - `Microsoft Learn Catalog API`를 통해 사용자의 요청에 맞는 강좌를 찾을 수 있도록 돕습니다
 - `Function Calling`을 사용해 사용자의 쿼리를 함수로 전달하고, API 요청을 보냅니다

먼저, 함수 호출을 왜 사용하는지부터 살펴보겠습니다:

print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # GPT가 함수 응답을 볼 수 있도록 새로운 응답을 받아옵니다

print(second_response.choices[0].message)


### 함수 호출이 필요한 이유

이 과정의 다른 강의를 완료했다면, 대형 언어 모델(LLM)을 사용하는 것이 얼마나 강력한지 이미 알고 있을 것입니다. 아마 그 한계점도 어느 정도 느끼셨을 겁니다.

함수 호출(Function Calling)은 Azure Open AI Service에서 제공하는 기능으로, 다음과 같은 한계를 극복하기 위해 만들어졌습니다:
1) 일관된 응답 형식 제공
2) 채팅 맥락에서 애플리케이션의 다른 데이터 소스를 활용할 수 있는 능력

함수 호출 기능이 도입되기 전에는, LLM의 응답이 구조화되어 있지 않고 일관성도 없었습니다. 개발자들은 각기 다른 응답 형태를 처리하기 위해 복잡한 검증 코드를 작성해야 했습니다.

사용자들은 "스톡홀름의 현재 날씨가 어때?"와 같은 질문에 답을 받을 수 없었습니다. 이는 모델이 학습된 시점의 데이터에만 제한되어 있었기 때문입니다.

아래 예시를 통해 이 문제를 좀 더 자세히 살펴보겠습니다:

학생 데이터베이스를 만들어서 각 학생에게 적합한 과정을 추천해주고 싶다고 가정해봅시다. 아래에는 비슷한 정보를 담고 있는 두 학생의 설명이 있습니다.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

이 데이터를 파싱하기 위해 LLM에 보내고자 합니다. 이후에는 이 데이터를 API로 전송하거나 데이터베이스에 저장하는 데 사용할 수 있습니다.

관심 있는 정보를 LLM에 안내하는 동일한 프롬프트 두 개를 만들어봅시다:


우리는 이것을 LLM에 보내 우리 제품에 중요한 부분을 파싱하도록 하고자 합니다. 따라서 LLM에 지시할 수 있도록 두 개의 동일한 프롬프트를 만들 수 있습니다.


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


이 두 프롬프트를 만든 후, 우리는 `openai.ChatCompletion`을 사용하여 LLM에 보낼 것입니다. 프롬프트는 `messages` 변수에 저장하고 역할은 `user`로 지정합니다. 이는 사용자가 챗봇에 메시지를 작성하는 것을 모방하기 위함입니다.


In [ ]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-07-01-preview"
  )

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']

: 

In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

프롬프트가 같고 설명도 비슷하지만, `Grades` 속성의 형식은 다르게 나올 수 있습니다.

위 셀을 여러 번 실행하면, 형식이 `3.7`일 수도 있고 `3.7 GPA`일 수도 있습니다.

이는 LLM이 프롬프트처럼 비구조화된 데이터를 입력받아, 역시 비구조화된 데이터를 반환하기 때문입니다. 데이터를 저장하거나 사용할 때 어떤 형식이 올지 예측할 수 있도록, 구조화된 형식이 필요합니다.

함수 호출 기능을 사용하면, 구조화된 데이터를 받을 수 있습니다. 함수 호출을 사용할 때 LLM이 실제로 함수를 호출하거나 실행하는 것은 아닙니다. 대신, LLM이 응답할 때 따라야 할 구조를 만들어줍니다. 그리고 이렇게 구조화된 응답을 활용해, 우리 애플리케이션에서 어떤 함수를 실행할지 결정할 수 있습니다.


![함수 호출 흐름도](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.ko.png)


### 함수 호출 사용 사례

**외부 도구 호출하기**  
챗봇은 사용자의 질문에 답변을 제공하는 데 매우 유용합니다. 함수 호출을 활용하면 챗봇이 사용자의 메시지를 받아 특정 작업을 수행할 수 있습니다. 예를 들어, 학생이 챗봇에게 "이 과목에 더 많은 도움이 필요하다고 교수님께 이메일 보내줘"라고 요청할 수 있습니다. 이 경우 `send_email(to: string, body: string)` 함수를 호출할 수 있습니다.

**API 또는 데이터베이스 쿼리 생성**  
사용자는 자연어로 정보를 요청하고, 이를 포맷된 쿼리나 API 요청으로 변환할 수 있습니다. 예를 들어, 교사가 "마지막 과제를 완료한 학생이 누구야?"라고 물으면 `get_completed(student_name: string, assignment: int, current_status: string)`라는 함수를 호출할 수 있습니다.

**구조화된 데이터 생성하기**  
사용자는 텍스트 블록이나 CSV 파일에서 중요한 정보를 추출하도록 LLM을 활용할 수 있습니다. 예를 들어, 학생이 평화 협정에 관한 위키피디아 글을 AI 플래시카드로 만들고 싶을 때 사용할 수 있습니다. 이때 `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)` 함수를 사용할 수 있습니다.


## 2. 첫 번째 함수 호출 만들기

함수 호출을 만드는 과정은 크게 3단계로 이루어집니다:
1. 함수 목록과 사용자 메시지를 포함하여 Chat Completions API를 호출합니다.
2. 모델의 응답을 읽고, 그에 따라 함수 실행이나 API 호출 등 필요한 작업을 수행합니다.
3. 함수의 응답을 Chat Completions API에 다시 전달하여, 그 정보를 바탕으로 사용자에게 보낼 답변을 생성합니다.


![함수 호출의 흐름](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.ko.png)


### 함수 호출의 요소

#### 사용자 입력

첫 번째 단계는 사용자 메시지를 만드는 것입니다. 이는 텍스트 입력값을 받아 동적으로 할당할 수도 있고, 여기서 값을 직접 지정할 수도 있습니다. Chat Completions API를 처음 사용하는 경우, 메시지의 `role`과 `content`를 정의해야 합니다.

`role`은 `system`(규칙 생성), `assistant`(모델), `user`(최종 사용자) 중 하나가 될 수 있습니다. 함수 호출을 위해서는 `user`로 지정하고, 예시 질문을 입력합니다.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### 함수 만들기

이제 함수와 그 함수의 매개변수를 정의해보겠습니다. 여기서는 `search_courses`라는 하나의 함수만 사용하지만, 여러 개의 함수를 만들 수도 있습니다.

**중요** : 함수는 시스템 메시지에 포함되어 LLM에 전달되며, 사용 가능한 토큰 수에 영향을 미칩니다.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**정의**

`name` - 호출하고자 하는 함수의 이름입니다.

`description` - 함수가 어떻게 동작하는지에 대한 설명입니다. 여기서는 구체적이고 명확하게 작성하는 것이 중요합니다.

`parameters` - 모델이 응답에서 생성해야 하는 값들과 그 형식의 목록입니다.

`type` - 속성에 저장될 데이터 타입입니다.

`properties` - 모델이 응답에 사용할 구체적인 값들의 목록입니다.

`name` - 모델이 포맷된 응답에서 사용할 속성의 이름입니다.

`type` - 이 속성의 데이터 타입입니다.

`description` - 특정 속성에 대한 설명입니다.

**선택 사항**

`required` - 함수 호출이 완료되기 위해 필요한 속성입니다.


### 함수 호출하기
함수를 정의한 후에는 이제 Chat Completion API를 호출할 때 해당 함수를 포함해야 합니다. 이를 위해 요청에 `functions`를 추가합니다. 이 경우에는 `functions=functions`로 작성합니다.

또한 `function_call`을 `auto`로 설정하는 옵션도 있습니다. 이렇게 하면 어떤 함수를 호출할지 직접 지정하지 않고, 사용자 메시지를 바탕으로 LLM이 적절한 함수를 선택하도록 할 수 있습니다.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

이제 응답을 살펴보고 형식이 어떻게 되어 있는지 확인해 봅시다:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

여기서 함수의 이름이 호출되고, 사용자 메시지에서 LLM이 함수의 인자에 맞는 데이터를 찾아낸 것을 볼 수 있습니다.


## 3. 함수 호출을 애플리케이션에 통합하기

LLM에서 포맷된 응답을 테스트한 후, 이제 이를 애플리케이션에 통합할 수 있습니다.

### 흐름 관리하기

이 기능을 애플리케이션에 통합하려면 다음 단계를 따라 진행해봅시다.

먼저, Open AI 서비스에 요청을 보내고, 응답 메시지를 `response_message`라는 변수에 저장합니다.


In [ ]:
response_message = response.choices[0].message

이제 Microsoft Learn API를 호출하여 강좌 목록을 가져오는 함수를 정의하겠습니다:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



최선의 방법으로, 먼저 모델이 함수를 호출하려는지 확인합니다. 그 다음, 사용 가능한 함수 중 하나를 생성하고 호출하려는 함수와 일치시키겠습니다.
이후 함수의 인자를 받아 LLM에서 받은 인자와 매핑합니다.

마지막으로, 함수 호출 메시지와 `search_courses` 메시지에서 반환된 값을 추가합니다. 이렇게 하면 LLM이 사용자에게 자연스러운 언어로
응답하는 데 필요한 모든 정보를 갖추게 됩니다.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## 코드 챌린지

잘 하셨습니다! Azure Open AI Function Calling 학습을 계속하려면 다음을 시도해 보세요: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst  
- 학습자가 더 많은 강좌를 찾을 수 있도록 함수에 더 많은 매개변수를 추가해 보세요. 사용 가능한 API 매개변수는 여기에서 확인할 수 있습니다:  
- 학습자의 모국어와 같은 추가 정보를 받아들이는 또 다른 함수 호출을 만들어 보세요  
- 함수 호출 및/또는 API 호출에서 적합한 강좌를 반환하지 않을 때 오류 처리를 구현해 보세요



---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있으나, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서(원어)가 공식적인 기준임을 유의해 주시기 바랍니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 본 번역 사용으로 인해 발생하는 오해나 오역에 대해 당사는 책임을 지지 않습니다.
